In [8]:
import torch
import torchvision.datasets as dsets
import torch.nn as nn
import torchvision.transforms as transforms

import torch.nn.init

In [16]:
torch.cuda.is_available()

True

In [10]:
#parameters -> 나중에 넣어도 상관없음
learning_rate = 0.001
training_epochs = 15
batch_size = 100  #batch_size는 100개가 된다

In [11]:
#MNIST dataset

mnist_train = dsets.MNIST(root='MNIST_data/',
                        train = True,
                        transform = transforms.ToTensor(),  # -> input data를 어떻게 변환할지
                                                            # 그냥 받아오면 Tensor value가 아니라서 mnist_data를 Tensor value로 전환
                        download = True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                        train = False,
                        transform = transforms.ToTensor(),  # tensor로 변환
                        download = True)


In [12]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size = batch_size,
                                          shuffle = True, # dataset을 섞은 상태로 던져준다.
                                          drop_last=True)

In [18]:
#model  -> cnn 모듈 만들기

class CNN(nn.Module):
    
    def __init__(self):  # 초기함수 -> 이거 빼먹으면 학습 안됨.
        super(CNN,self).__init__()
        self.layer1 = nn.Sequential(   # nn.Sequential은 그냥 치고 내가 원하는걸 만들어서 넣는 방식으로 사용
            nn.Conv2d(1,32,kernel_size=3, stride=1, padding=1), # layer 한 개 쌓임
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32,64,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.fc = nn.Linear(7*7*64, 10, bias = True) 
        torch.nn.init.xavier_uniform_(self.fc.weight)
        
    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        
        out = out.view(out.size(0),-1)
        out = self.fc(out)
        
        return out

In [19]:
model = CNN().to(device)

value

In [21]:
model 
#layer1, layer2, fully_connected layer을 통과할 수 있는 layer들을 가지고 있는 모델이 생성

CNN(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=3136, out_features=10, bias=True)
)

In [24]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

In [26]:
#training
total_batch = len(data_loader) # batch_size가 몇인지를 알 수 있음 (전체의 batch가 몇인지)

for epoch in range(training_epochs):
    avg_cost = 0 #loss를 담을 것
    
    for X, Y in data_loader: # X = input image, Y = label
        # CUDA 연산을 진행하려면 torch.Tensor가 아니라 torch.cuda.Tensor가 되어있어야함
        # 없으면 아예 학습이 안된다.
        # input data, label data를 device에 맞춰서 설정해주는 작업
        X = X.to(device)
        Y = Y.to(device)   
        
        optimizer.zero_grad()
        hypothesis = model(X) # model에 input값 넣은 출력값
        
        # model에 input 값을 집어넣고 나온 cost는 cross_entropy로 계산
        cost = criterion(hypothesis, Y) # 가설과 실제 label 사이 차이 계산
        cost.backward()
        optimizer.step() # cost.backward()값을 토대로 model을 학습시키는 과정
        
        avg_cost += cost / total_batch
        
    
    print('[Epoch:{}] cost = {}'.format(epoch+1,avg_cost))
    
print('Learning Finished!')

[Epoch:1] cost = 0.007500140927731991
[Epoch:2] cost = 0.0057548354379832745
[Epoch:3] cost = 0.00433866074308753
[Epoch:4] cost = 0.004347255919128656
[Epoch:5] cost = 0.004201987292617559
[Epoch:6] cost = 0.004209483973681927
[Epoch:7] cost = 0.004311944358050823
[Epoch:8] cost = 0.00291151599958539
[Epoch:9] cost = 0.002519402652978897
[Epoch:10] cost = 0.004405844956636429
[Epoch:11] cost = 0.0017666355706751347
[Epoch:12] cost = 0.0026767386589199305
[Epoch:13] cost = 0.00252183573320508
[Epoch:14] cost = 0.0006437206757254899
[Epoch:15] cost = 0.0031178344506770372
Learning Finished!


In [27]:
with torch.no_grad(): # 학습 x , test o
    # 아까 mnist_test에서 불러옴
    X_test = mnist_test.test_data.view(len(mnist_test),1,28,28).float().to(device) #len()은 전체 값을 불러서 한 번에 집어넣으려고 펼친 것
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test) # 한 번에 집어넣음
    correct_prediction = torch.argmax(prediction, 1) == Y_test # Y_test와 비교
    accuracy = correct_prediction.float().mean() # 평균
    print('Accuracy:',accuracy.item())

C:\Users\user\.conda\envs\deeprunning2\lib\site-packages\torchvision\datasets\mnist.py:63: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\Users\user\.conda\envs\deeprunning2\lib\site-packages\torchvision\datasets\mnist.py:53: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.9856999516487122
